In [1]:
import numpy as np
import fasttext
import pandas as pd
import random
import itertools
from resources.tokTT import CommentTokenizer as CT
from resources.basicIO import InputOutput as IO
from resources.basicIO import InputOutput as IO
from resources.filterLang import FilterLanguage as FL
from resources.fasttext_transformer import classify
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neural_network import MLPClassifier
import fasttext
import pandas as pd
from scipy import spatial
from scipy.spatial import distance
from sklearn import svm
from sklearn import metrics
from sklearn.linear_model import SGDClassifier
#from sklearn.linear_model import LassoLars
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
import copy
import scipy
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import validation_curve
import gc


[nltk_data] Downloading package wordnet to C:\Users\AJAY
[nltk_data]     BISWAS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Load Fasttext Unsupervised Model

In [2]:
model_2 = fasttext.load_model("models/ft_unsupervised_N_2.bin")
model_3 = fasttext.load_model("models/ft_unsupervised_N_3.bin")

### Make dataframes

In [3]:
# random sample
text = IO.load_csv_col('datasets/random_sample.csv', 'comment')
text_labels = IO.load_csv_col('datasets/random_sample.csv', 'label')
text_labels = list(map(str, map(int, text_labels)))
text_TK = [CT.tokenize(x) for x in text]

df_dict = {'raw_comment': text,
           'tokenized_comment': text_TK, 'label': text_labels}
df_sample = pd.DataFrame(df_dict)
df_sample.to_csv('datasets/random_sample_data.csv', index=False)


### Remove Unnecessary Comments

In [4]:
# remove unnecessary comments
df_sample = df_sample.drop([x for x in range(len(df_sample)) if int(
    df_sample['label'][x]) != 0 and int(df_sample['label'][x]) != 1])

In [5]:
df_sample.tail()


,raw_comment,tokenized_comment,label
4477,all the flags are sikh religious flags. they d...,all the flag be sikh religious flag they do no...,1
4478,Salute the farmers,salute the farmer,0
4479,Let's also remember how many farmers lost thei...,let be also remember how many farmer lose thei...,0
4480,ABP news you guys dont have heart 8 farmers go...,abp news you guy dont have heart 8 farmer get ...,0
4481,These new laws actually give the power to the ...,these new law actually give the power to the f...,1


In [6]:
df_sample['label'].value_counts()


0    1847
1    1407
Name: label, dtype: int64

### Train test split

In [7]:

X_train, X_test, y_train, y_test = train_test_split(df_sample['tokenized_comment'],
                                                    df_sample['label'], test_size=0.2,
                                                    random_state=37,
                                                    stratify=df_sample['label'])


In [8]:
print('X_train: ' ,len(X_train))
print('X_test: ' ,len(X_test))

X_train:  2603
X_test:  651


### Classifier

In [9]:
classifiers = {
    "SVM": make_pipeline(StandardScaler(), svm.SVC(probability=True, kernel='rbf')),
    "KNN": KNeighborsClassifier(n_neighbors=19),
    "LR": make_pipeline(StandardScaler(), LogisticRegression(solver='lbfgs', penalty='l2', C=0.05, max_iter=10000, random_state=2)),
    "MLP": make_pipeline(StandardScaler(), MLPClassifier(max_iter=1000, activation='logistic'))
}


In [10]:
# Logistic Regression
LR_Normal = classify(model_2, classifiers["LR"], X_train, y_train)
# SVM
SVM_Normal = classify(model_2, classifiers["SVM"], X_train, y_train)
# KNN
KNN_Normal = classify(model_2, classifiers["KNN"], X_train, y_train)
# MLP
MLP_Normal = classify(model_2, classifiers["MLP"], X_train, y_train)


# Logistic Regression
LR_Normal3 = classify(model_3, classifiers["LR"], X_train, y_train)
# SVM
SVM_Normal3 = classify(model_3, classifiers["SVM"], X_train, y_train)
# KNN
KNN_Normal3 = classify(model_3, classifiers["KNN"], X_train, y_train)
# MLP
MLP_Normal3 = classify(model_3, classifiers["MLP"], X_train, y_train)

models = []
models.append(('LR Normal N=2', LR_Normal))
models.append(('SVM Normal N=2', SVM_Normal))
models.append(('KNN Normal N=2', KNN_Normal))
models.append(('MLP Normal N=2', MLP_Normal))
models.append(('LR Normal N=3', LR_Normal3))
models.append(('SVM Normal N=3', SVM_Normal3))
models.append(('KNN Normal N=3', KNN_Normal3))
models.append(('MLP Normal N=3', MLP_Normal3))


In [11]:
file = open('results/output_normal.txt', 'w+')
file.close()

outfile = open("results/output_normal.txt", "a")
for i, v in models:
    print(i)
    accuracy = metrics.accuracy_score(y_test, v.predict(X_test))
    confusion_matrix = metrics.confusion_matrix(y_test, v.predict(X_test))
    print('========= {} Model Test Results ==========='.format(i), file=outfile)
    print(' ', file=outfile)
    print("Model Accuracy:" "\n", accuracy, file=outfile)
    print(accuracy)
    print(' ', file=outfile)
    print("Confusion matrix:" "\n", confusion_matrix, file=outfile)
    print(' ', file=outfile)
outfile.close()


LR Normal N=2
0.7511520737327189
SVM Normal N=2
0.7895545314900153
KNN Normal N=2
0.7112135176651305
MLP Normal N=2
0.7680491551459293
LR Normal N=3
0.7588325652841782
SVM Normal N=3
0.7880184331797235
KNN Normal N=3
0.7112135176651305
MLP Normal N=3
0.7788018433179723


### Active Learning

In [12]:
X_seed, X_expand, y_seed, y_expand = train_test_split(X_train,
                                                      y_train, test_size=0.99,
                                                      random_state=40,
                                                      stratify=y_train)


In [13]:
print('X_seed: ',len(X_seed))
print('X_expand: ',len(X_expand))

X_seed:  26
X_expand:  2577


In [14]:
y_seed.value_counts()


0    15
1    11
Name: label, dtype: int64

In [15]:
df_dict = {'seed_tokenized': X_seed,
           'label': y_seed}
df_seed = pd.DataFrame(df_dict)
df_seed.to_csv('datasets/seed_data.csv', index=False)

df_dict = {'expansion_tokenized': X_expand,
           'label': y_expand}
df_expand = pd.DataFrame(df_dict)
df_seed.to_csv('datasets/expand_data.csv', index=False)


### Cosine Similarity And Nearest Neighbors

In [16]:
def score(model, line, k):
    """ Returns a vector containing nearest neighbor scores w.r.t. all
        words in the model """
    # words contains all the words in the corpus
    lst1 = model.get_nearest_neighbors(line, k)
    v1 = []
    l1 = [x[1] for x in lst1]
    l10 = [x[0] for x in lst1]
    for i in range(len(model.words)):
        try:
            v1.append(l10[l1.index(model.words[i])])
        except:
            v1.append(0)
    return v1


def NN(model: fasttext.FastText._FastText, line: str, K):
    """ Returns k fasttext nearest neighbors of a given string """
    return model.get_nearest_neighbors(line, k=K)


def get_NN(model: fasttext.FastText._FastText, lines: list, k: int):
    """ Returns k nearest neighbor scores of multiple strings"""
    scores = []
    for line in lines:
        scores.append(score(model, line, k))
    return scores

def cos_sim(a: np.array, b: np.array):
    """ Returns cosine similarity of two 1d arrays """
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    if(norm_a * norm_b == 0.0):
        return dot_product / (norm_a * norm_b + 0.001)
    return dot_product / (norm_a * norm_b)


def sim(x: np.array, y: np.array, sim_type: str):
    if(sim_type == 'cosine_sim'):
        return cos_sim(x, y)


def sim_matrix(A: np.array, B: np.array, sim_type: str):
    """ find similarity score matrix between A and B. 
        A,B: 2d matrix of embeddings/nearest neighbor scores.
        sim_type: String denoting type of similarity.
    """
    m, p = A.shape
    p, n = B.shape
    C = np.zeros((m, n))
    for i in range(m):
        for j in range(n):
            C[i][j] = sim(A[i, :], B[:, j], sim_type)
    return C


### Expansion Code (Random Sampling)

In [17]:
def Expand_R(model: fasttext.FastText._FastText, 
             seed_set_tokenised: list, 
             seed_set_label: list, 
             expansion_tokenised: list, 
             expansion_set_labels: list, 
             batch_size: int, 
             k_neighbors: int, 
             random_rate: float):

    """
    Takes seed set and and expands the set using expansion_tokenised.
    Batch size: no. of texts to be inserted in one go
    k_neighbors: no. of neighbors for getting cosine similarity
    random_rate: fraction of amount taken for random sampling
    """
             
    seed_TK = copy.deepcopy(seed_set_tokenised)
    seed_labels = copy.deepcopy(seed_set_label)
    count = len(expansion_set_labels)
    M = np.arange(0, count, batch_size)
    cnt = int(random_rate * batch_size)
    
    expansion_predicted_labels = []
    expansion_true_labels = []

    for i in range(1, len(M)):

        #print(M[i], end=' ')

        # select batchwise expansion text
        exp_TK = expansion_tokenised[M[i-1]:M[i]]
        exp_labels = expansion_set_labels[M[i-1]:M[i]]

        # nearest neighbors
        seed_NN = get_NN(model, seed_TK, k_neighbors)
        exp_NN = get_NN(model, exp_TK, k_neighbors)

        A = np.array(seed_NN)
        B = np.array(exp_NN).T
        C = sim_matrix(A, B, "cosine_sim")

        # find rowwise (seed) index of highest similarity
        Y_ind = np.argmax(C, axis=0)
        # get labels
        Y = [seed_labels[x] for x in Y_ind]

        if(random_rate == 0.0):
            # no random sampling
            pass
        else:
            # random sampling
            Y_r = random.sample(range(0,len(Y)), cnt)
            for j in Y_r:
                Y[j] = exp_labels[j]

        # calc. expansion accuracy
        expansion_predicted_labels.extend(Y)
        expansion_true_labels.extend(exp_labels)

        # expand seed set
        seed_labels.extend(Y)
        seed_TK.extend(exp_TK)

    return seed_TK, seed_labels, expansion_true_labels, expansion_predicted_labels


In [19]:

print('model ', 'batch_size, ', 'Random Rate, ' 'expanded_size, ', 'outp_algorithm, ', 'c_accuracy')

models = [('fasttext', 'models/ft_unsupervised_N_2.bin'),
          ('fasttext', 'models/ft_unsupervised_N_3.bin')]

batch_sizes = [20]
random_rates = [0.2]
outp_algorithms = ['LR','SVM','KNN','MLP']
big_results_r = []
big_confusion_r = []

for md in models:
    mdl = None
    if(md[0] == 'fasttext'):
        mdl = fasttext.load_model(md[1])
    
    for batch_size in batch_sizes:
        for random_rate in random_rates:
            seed_TK, seed_labels, _tk, _labels = Expand_R(mdl,
                                                list(
                                                    X_seed),
                                                list(
                                                    y_seed),
                                                list(
                                                    X_expand),
                                                list(
                                                    y_expand),
                                                batch_size,
                                                40,
                                                random_rate)
                                                                              
            for outp_algorithm in outp_algorithms:    

                alg2 = classifiers[outp_algorithm]

                # classify
                c_model = classify(mdl, alg2, seed_TK, seed_labels)
                y_predict = c_model.predict(X_test)
                c_accuracy = metrics.accuracy_score(y_test, y_predict)
                c_prfsw = list(precision_recall_fscore_support(y_test, y_predict, average='weighted'))
                c_prfsmi = list(precision_recall_fscore_support(y_test, y_predict, average='micro'))

                l1 = [md[1], batch_size, random_rate, len(seed_TK), outp_algorithm, c_accuracy]
                l1.extend(c_prfsw)
                l1.extend(c_prfsmi)

                big_results_r.append(l1)
                big_confusion_r.append(metrics.confusion_matrix(y_test, c_model.predict(X_test)))
                print(md[1], ',', batch_size, ',', random_rate, ',', len(seed_TK), ',', outp_algorithm, ',', c_accuracy)
    del mdl
    gc.collect()



model  batch_size,  Random Rate, expanded_size,  outp_algorithm,  c_accuracy


models/ft_unsupervised_N_2.bin , 20 , 0.2 , 2586 , LR , 0.7096774193548387
models/ft_unsupervised_N_2.bin , 20 , 0.2 , 2586 , SVM , 0.706605222734255
models/ft_unsupervised_N_2.bin , 20 , 0.2 , 2586 , KNN , 0.6712749615975423
models/ft_unsupervised_N_2.bin , 20 , 0.2 , 2586 , MLP , 0.6989247311827957


models/ft_unsupervised_N_3.bin , 20 , 0.2 , 2586 , LR , 0.6897081413210445
models/ft_unsupervised_N_3.bin , 20 , 0.2 , 2586 , SVM , 0.7035330261136713
models/ft_unsupervised_N_3.bin , 20 , 0.2 , 2586 , KNN , 0.6666666666666666
models/ft_unsupervised_N_3.bin , 20 , 0.2 , 2586 , MLP , 0.6697388632872504


In [22]:

res_df = pd.DataFrame(big_results_r)
res_df.to_csv("results/big_results_r3.csv", index=False, header=False)

print(np.array(big_confusion_r))


[[[283  87]
  [102 179]]

 [[293  77]
  [114 167]]

 [[236 134]
  [ 80 201]]

 [[276  94]
  [102 179]]

 [[277  93]
  [109 172]]

 [[288  82]
  [111 170]]

 [[239 131]
  [ 86 195]]

 [[266 104]
  [111 170]]]


### Uncertainty Sampling

In [ ]:
def Expand_U(model: fasttext.FastText._FastText, 
             algorithm: object, 
             seed_set_tokenised: list, 
             seed_set_label: list, 
             expansion_tokenised: list,
             expansion_set_labels: list, 
             batch_size: int,
             countMax: int):
    """ Uncertainty sampling.
    Expand seed set using expansion_set based on lowest confidance scores.
    max_threshold: max. probability for uncertainty selection """

    seed_TK = copy.deepcopy(seed_set_tokenised)
    seed_labels = copy.deepcopy(seed_set_label)
    count = len(expansion_set_labels)
    M = np.arange(0, count, batch_size)

    # exp_TK_certain will be the list of comments having high proba score
    exp_TK_certain = []
    exp_TK_certain_labels = []


    for i in range(1, len(M)):

        #print(M[i], end=' ')

        exp_TK = expansion_tokenised[M[i-1]:M[i]]
        exp_labels = expansion_set_labels[M[i-1]:M[i]]

        # take A as training and B as test and store probs in C
        small_model = classify(model, algorithm, seed_TK, seed_labels)
        # store classwise prob. scores
        C = small_model.predict_proba(exp_TK)
        # Uncertainty sampling scores
        C_abs_diff = [(abs(x[0] - x[1])) for x in C]

        y_predict = small_model.predict(X_test)
        c_accuracy = metrics.accuracy_score(y_test, y_predict)
        c_prfsw = list(precision_recall_fscore_support(
            y_test, y_predict, average='weighted'))
        c_prfsmi = list(precision_recall_fscore_support(
            y_test, y_predict, average='micro'))
        
        l1 = []
        l1.append(c_accuracy)
        l1.extend(c_prfsw[0:3])
        l1.extend(c_prfsmi[0:3])

        # Sort lists in ascending order of probabilities from C_abs_diff
        sorted_lists = sorted(zip(exp_labels, exp_TK, C, C_abs_diff), key=lambda x: x[3])
        exp_labels, exp_TK, C_sorted, score = [[x[i] for x in sorted_lists] for i in range(4)]

        Y_uncertain = []
        exp_TK_uncertain = []
        for j in range(len(C_sorted)):
            #max_value = max(C_sorted[j])
            #max_index = str(np.argmax(C_sorted[j]))

            # label the comments whose score is less than threshold
            if(j < countMax):

                exp_TK_uncertain.append(exp_TK[j])
                Y_uncertain.append(exp_labels[j])
            else:
                exp_TK_certain.append(exp_TK[j])
                exp_TK_certain_labels.append(exp_labels[j])

        # expand the seed set
        seed_labels.extend(Y_uncertain)
        seed_TK.extend(exp_TK_uncertain)


    return seed_TK, seed_labels, exp_TK_certain, exp_TK_certain_labels


### Automatic 

In [ ]:
#print('model ', 'algorithm, ', 'batch_size, ', 'best_amount, ', 'expanded_size, ', 'outp_algorithm, ', 'c_accuracy')

models = [('fasttext', 'models/ft_unsupervised_N_2.bin'),
          ('fasttext', 'models/ft_unsupervised_N_3.bin')
          ]
algorithms = ['LR', 'SVM', 'KNN', 'MLP']
batch_sizes = [20]
best_amounts = [5]
big_results = []
big_confusion = []
for md in models:
    mdl = None
    if(md[0] == 'fasttext'):
        mdl = fasttext.load_model(md[1])

    for algorithm in algorithms:
        alg = classifiers[algorithm]

        for batch_size in batch_sizes:

            for best_amount in best_amounts:

                X_uncertain_exp_u, y_uncertain_exp_u, X_certain_exp_u, y_certain_exp_u = Expand_U(mdl,
                                                                                        alg,
                                                                                        list(
                                                                                            X_seed),
                                                                                        list(
                                                                                            y_seed),
                                                                                        list(
                                                                                            X_expand),
                                                                                        list(
                                                                                            y_expand),
                                                                                        batch_size,
                                                                                        best_amount
                                                                                        )

                # classify
                c_model = classify(mdl, alg, X_uncertain_exp_u, y_uncertain_exp_u)
                y_predict = c_model.predict(X_test)
                c_accuracy = metrics.accuracy_score(y_test, y_predict)
                c_prfsw = list(precision_recall_fscore_support(y_test, y_predict, average='weighted'))
                c_prfsmi = list(precision_recall_fscore_support(y_test, y_predict, average='micro'))

                l1 = [md[1], algorithm, batch_size, best_amount, len(
                    X_uncertain_exp_u), c_accuracy]
                l1.extend(c_prfsw)
                l1.extend(c_prfsmi)


                big_results.append(l1)
                big_confusion.append(metrics.confusion_matrix(y_test, c_model.predict(X_test)))
                #print(md[1], ',', algorithm, ',', batch_size, ',', best_amount, ',', len(X_uncertain_exp_u), ',', outp_algorithm, ',', c_accuracy)
    del mdl
    gc.collect()


In [ ]:
res_df = pd.DataFrame(big_results)
res_df.to_csv("results/big_results3.csv", index=False, header=False)


In [ ]:
print(np.array(big_confusion))

[[[301  69]
  [ 65 216]]

 [[326  44]
  [ 69 212]]

 [[239 131]
  [ 65 216]]

 [[313  57]
  [ 59 222]]

 [[298  72]
  [ 74 207]]

 [[314  56]
  [ 58 223]]

 [[249 121]
  [ 76 205]]

 [[312  58]
  [ 61 220]]]
